In [1]:
import numpy as np

# 定義參數
L1 = 100
L2 = 105
L3 = 98
L4 = 245
IN_theta = [0, -45, 10, 10, 0]

# DH 參數
C_a = [0, 0, L2, L3, 0, 0, 0]
C_d = [0, L1, 0, 0, 0, 0, L4]
C_alpha = [0, -90, 0, 0, -90, 0, 0]
C_theta = [0, IN_theta[0], IN_theta[1], IN_theta[2], IN_theta[3] - 90, IN_theta[4], 0]

# 初始化結果矩陣為 4x4 單位矩陣
T_result = np.eye(4)

# 定義角度轉弧度函數
def cosd(theta):
    return np.cos(np.deg2rad(theta))

def sind(theta):
    return np.sin(np.deg2rad(theta))

# 計算每個T[i]並累積到T_result
for i in range(6):
    T = np.array([
        [cosd(C_theta[i+1]), -sind(C_theta[i+1]), 0, C_a[i]],
        [sind(C_theta[i+1]) * cosd(C_alpha[i]), cosd(C_alpha[i]) * cosd(C_theta[i+1]), -sind(C_alpha[i]), -sind(C_alpha[i]) * C_d[i+1]],
        [sind(C_theta[i+1]) * sind(C_alpha[i]), cosd(C_theta[i+1]) * sind(C_alpha[i]), cosd(C_alpha[i]), cosd(C_alpha[i]) * C_d[i+1]],
        [0, 0, 0, 1]
    ])
    
    T_result = np.dot(T_result, T)  # 累積每次的變換矩陣

# 打印最終變換矩陣
print("Transformation Matrix T_result:")
print(T_result)


Transformation Matrix T_result:
[[-4.22618262e-01  5.54953465e-17  9.06307787e-01  3.76568520e+02]
 [-5.54953465e-17 -1.00000000e+00  3.53544349e-17  6.73667371e-16]
 [ 9.06307787e-01 -3.53544349e-17  4.22618262e-01  3.33998177e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [3]:
import numpy as np
import math

# 机械臂关节长度
l1 = 100
l2 = 105
l3 = 98
l4 = 245
IN_theta = [0, -45, 20, 90, 0]

# DH 参数
C_a = [0, 0, l2, l3, 0, 0, 0]
C_d = [0, l1, 0, 0, 0, 0, l4]
C_alpha = [0, -90, 0, 0, -90, 0, 0]
C_theta = [0, IN_theta[0], IN_theta[1], IN_theta[2], IN_theta[3] - 90, IN_theta[4], 0]

# 正向运动学计算
T_target = np.eye(4)  # 初始化结果为单位矩阵
def calculate_joint_angles(x, y, z):
    # 机械臂关节长度
    l1 = 100
    l2 = 105
    l3 = 98
    l4 = 245

    # 计算 theta1
    theta1 = np.arctan2(y, x)

    # 在 x-y 平面投影中的半径
    r = np.sqrt(x**2 + y**2)
    # 初始化找到解的标志
    found_solution = False

    # 循环尝试不同的 a 值
    for a_deg in range(-90, 91):
        a = np.radians(a_deg)
        # 计算 xc 和 zc
        xc = r - np.cos(a) * l4
        zc = z - np.sin(a) * l4

        # 计算 lac_sq
        lac_sq = xc**2 + (zc - l1)**2

        # 检查是否在范围内
        if lac_sq > (l2 + l3)**2 or lac_sq < (l2 - l3)**2:
            continue

        # 计算 jbac 和 jcac_prime
        jbac = np.arccos((l2**2 + lac_sq - l3**2) / (2 * l2 * np.sqrt(lac_sq)))
        jcac_prime = np.arctan2(zc - l1, xc)

        # 计算 theta2
        theta2 = -jbac - jcac_prime

        # 计算 theta3
        theta3 = np.pi - np.arccos((l2**2 + l3**2 - lac_sq) / (2 * l2 * l3))

        # 检查是否有虚数部分
        if np.isreal(theta2) and np.isreal(theta3):
            # 计算 theta4
            theta4 = - theta2 - theta3 - a

            # 标记已找到解
            found_solution = True

            break  # 结束循环

    if found_solution:
        # 返回关节角度数组
        C_theta1 = [0, np.degrees(theta1), np.degrees(theta2), np.degrees(theta3), np.degrees(theta4) - 90, 0, 0]
        print(f'Theta1: {np.degrees(theta1)} degrees')
        print(f'Theta2: {np.degrees(theta2)} degrees')
        print(f'Theta3: {np.degrees(theta3)} degrees')
        print(f'Theta4: {np.degrees(theta4)} degrees')
    else:
        print('找不到合适的逆解')
        C_theta1 = np.full(7, np.nan)  # 返回 NaN 表示未找到解

    return C_theta1

for i in range(6):
    theta = np.radians(C_theta[i + 1])
    T = np.array([[np.cos(theta), -np.sin(theta), 0, C_a[i]],
                  [np.sin(theta) * np.cos(np.radians(C_alpha[i])), np.cos(theta) * np.cos(np.radians(C_alpha[i])), -np.sin(np.radians(C_alpha[i])), -np.sin(np.radians(C_alpha[i])) * C_d[i + 1]],
                  [np.sin(theta) * np.sin(np.radians(C_alpha[i])), np.cos(theta) * np.sin(np.radians(C_alpha[i])), np.cos(np.radians(C_alpha[i])), np.cos(np.radians(C_alpha[i])) * C_d[i + 1]],
                  [0, 0, 0, 1]])
    
    T_target = np.dot(T_target, T)  # 乘以每个 T[i]

# 逆运动学计算
C_theta1 = calculate_joint_angles(T_target[0, 3], T_target[1, 3], T_target[2, 3])
T_result = np.eye(4)  # 重新初始化结果为单位矩阵

for i in range(6):
    theta = np.radians(C_theta1[i + 1])
    T = np.array([[np.cos(theta), -np.sin(theta), 0, C_a[i]],
                  [np.sin(theta) * np.cos(np.radians(C_alpha[i])), np.cos(theta) * np.cos(np.radians(C_alpha[i])), -np.sin(np.radians(C_alpha[i])), -np.sin(np.radians(C_alpha[i])) * C_d[i + 1]],
                  [np.sin(theta) * np.sin(np.radians(C_alpha[i])), np.cos(theta) * np.sin(np.radians(C_alpha[i])), np.cos(np.radians(C_alpha[i])), np.cos(np.radians(C_alpha[i])) * C_d[i + 1]],
                  [0, 0, 0, 1]])
    
    T_result = np.dot(T_result, T)  # 乘以每个 T[i]

# 输出结果
print('T_target 矩阵：')
print(T_target)
print('T_result 矩阵：')
print(T_result)



Theta1: 4.623960064192537e-15 degrees
Theta2: -45.00000000000001 degrees
Theta3: 20.000000000000053 degrees
Theta4: 89.99999999999994 degrees
T_target 矩阵：
[[ 9.06307787e-01  2.58779051e-17  4.22618262e-01  2.66605849e+02]
 [-2.58779051e-17 -1.00000000e+00  1.16727686e-16  2.15159792e-14]
 [ 4.22618262e-01 -1.16727686e-16 -9.06307787e-01 -6.38260615e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
T_result 矩阵：
[[ 9.06307787e-01  1.06581233e-16  4.22618262e-01  2.66605849e+02]
 [ 4.72641492e-17 -1.00000000e+00  1.50834387e-16  4.30319584e-14]
 [ 4.22618262e-01 -1.16727686e-16 -9.06307787e-01 -6.38260615e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [ ]:
import numpy as np
from sympy import symbols, Eq, solve, cos, sin, pi

# 机械臂参数
L1 = 100
L2 = 105
L3 = 98
L4 = 245
IN_theta = [0, -45, 30, 10, 0]

# Denavit-Hartenberg 参数
C_a = [0, 0, L2, L3, 0, 0, 0]
C_d = [0, L1, 0, 0, 0, 0, L4]
C_alpha = [0, -90, 0, 0, -90, 0, 0]
C_theta = [0, IN_theta[0], IN_theta[1], IN_theta[2], IN_theta[3] - 90, IN_theta[4], 0]

# 初始化目标矩阵
T_target = np.eye(4)

# 角度转弧度函数
def deg2rad(deg):
    return deg * np.pi / 180

# 计算正向运动学
def dh_transform(theta, d, a, alpha):
    theta = deg2rad(theta)
    alpha = deg2rad(alpha)
    return np.array([
        [np.cos(theta), -np.sin(theta), 0, a],
        [np.sin(theta)*np.cos(alpha), np.cos(theta)*np.cos(alpha), -np.sin(alpha), -np.sin(alpha)*d],
        [np.sin(theta)*np.sin(alpha), np.cos(theta)*np.sin(alpha), np.cos(alpha), np.cos(alpha)*d],
        [0, 0, 0, 1]
    ])

for i in range(6):
    T = dh_transform(C_theta[i+1], C_d[i+1], C_a[i], C_alpha[i])
    T_target = np.dot(T_target, T)

# 提取目标位置
x_target = T_target[0, 3]
y_target = T_target[1, 3]
z_target = T_target[2, 3]

# 初始化最优解
best_theta = []
min_error = float('inf')

# 逆运动学求解
theta2, theta3, theta4 = symbols('theta2 theta3 theta4')
for apha1 in range(-90, 91):  # L4与x轴的夹角，角度步长为1度
    eq1 = L4 * np.cos(deg2rad(apha1)) + L3 * cos(deg2rad(apha1 + theta4)) + L2 * cos(deg2rad(theta2)) - x_target
    eq2 = L1 + L4 * np.sin(deg2rad(apha1)) + L3 * sin(deg2rad(apha1 + theta4)) + L2 * sin(deg2rad(theta2)) - z_target
    eq3 = theta2 - (apha1 + theta3 + theta4)
    
    # 求解方程组
    solutions = solve([Eq(eq1, 0), Eq(eq2, 0), Eq(eq3, 0)], (theta2, theta3, theta4), dict=True)
    
    for sol in solutions:
        theta2_val = sol[theta2]
        theta3_val = sol[theta3]
        theta4_val = sol[theta4]
        
        # 计算误差
        x_calc = L4 * np.cos(deg2rad(apha1)) + L3 * np.cos(deg2rad(apha1 + theta4_val)) + L2 * np.cos(deg2rad(theta2_val))
        z_calc = L1 + L4 * np.sin(deg2rad(apha1)) + L3 * np.sin(deg2rad(apha1 + theta4_val)) + L2 * np.sin(deg2rad(theta2_val))
        error = np.sqrt((x_calc - x_target)**2 + (z_calc - z_target)**2)

        # 更新最优解
        if error < min_error:
            min_error = error
            best_theta = [0, theta2_val, theta3_val, theta4_val, 0, 0]

# 输出结果
if not best_theta:
    print('无法找到满足条件的解。')
else:
    print('最优解：')
    print(f'theta1: {best_theta[0]:.2f}')
    print(f'theta2: {theta2_val:.2f}')
    print(f'theta3: {theta3_val:.2f}')
    print(f'theta4: {theta4_val:.2f}')
    print(f'theta5: {best_theta[4]:.2f}')
    print(f'theta6: {best_theta[5]:.2f}')
    print(f'最小误差: {min_error:.2f}')
    print(f'x_calc: {x_calc:.2f}')
    print(f'z_calc: {z_calc:.2f}')
